# Homework 05

## 請參考鐵達尼號生存分析.ipynb，用 Keras 建構自己的模型做預測，並將結果上傳至 kaggle。

## 在這個作業裡面，你需要：

## 用 Keras 建構模型來預測 Titanic 資料集的結果。
## 比較 Homework03 的結果，看看哪一個結果比較好。
## 將相關資料推上 GitHub，並將網址填在助教開的 Google Spreadsheet 上。

# 載入模組

In [39]:
import pandas as pd
import numpy as np
import math
import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout


# 讀資料

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

# 找缺失值

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
family         891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


# 分析資料型態

In [4]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,family,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,0.904602,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,1.613459,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,1.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,10.000000,512.329200


# 加入新變數

In [5]:
test['family'] = test['Parch'] + test['SibSp']

In [6]:
train['family'] = train['Parch'] + train['SibSp']

In [10]:
#train['n01'] = train['Sex'] * 10 + train['Embarked']

In [11]:
#test['n01'] = test['Sex'] * 10 + test['Embarked']

# 特徵選取

In [51]:
selected_features = ['Pclass', 'Age', 'family','Parch','SibSp', 'Sex']

In [52]:
X_train = train[selected_features]

In [53]:
y_train = train['Survived']

In [54]:
X_test = test[selected_features]

# 處理缺值

In [55]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass    891 non-null int64
Age       714 non-null float64
family    891 non-null int64
Parch     891 non-null int64
SibSp     891 non-null int64
Sex       891 non-null object
dtypes: float64(1), int64(4), object(1)
memory usage: 41.8+ KB


In [56]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass    418 non-null int64
Age       332 non-null float64
family    418 non-null int64
Parch     418 non-null int64
SibSp     418 non-null int64
Sex       418 non-null object
dtypes: float64(1), int64(4), object(1)
memory usage: 19.7+ KB


In [57]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass    891 non-null int64
Age       714 non-null float64
family    891 non-null int64
Parch     891 non-null int64
SibSp     891 non-null int64
Sex       891 non-null object
dtypes: float64(1), int64(4), object(1)
memory usage: 41.8+ KB


In [58]:
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
#X_train['Embarked'].fillna('S', inplace=True)
X_test['Age'].fillna(X_train['Age'].mean(), inplace=True)
X_test['Fare'].fillna(X_train['Fare'].mean(), inplace=True)

C:\Users\x555l\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


KeyError: 'Fare'

# 文字去轉換成可分析

In [59]:
X_train = pd.get_dummies(X_train)

In [60]:
X_test = pd.get_dummies(X_test)

In [61]:
X_test.head()

,Pclass,Age,family,Parch,SibSp,Sex_female,Sex_male
0,3,34.5,0,0,0,0,1
1,3,47.0,1,0,1,1,0
2,2,62.0,0,0,0,0,1
3,3,27.0,0,0,0,0,1
4,3,22.0,2,1,1,1,0


In [62]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass        418 non-null int64
Age           418 non-null float64
family        418 non-null int64
Parch         418 non-null int64
SibSp         418 non-null int64
Sex_female    418 non-null uint8
Sex_male      418 non-null uint8
dtypes: float64(1), int64(4), uint8(2)
memory usage: 17.2 KB


In [63]:
model = Sequential()

In [64]:
model.add(Dense(units=64, activation='relu',kernel_initializer = 'uniform', input_dim=6))
model.add(Dense(units=10, activation='relu',kernel_initializer = 'uniform'))
model.add(Dense(units=1, activation='sigmoid',kernel_initializer = 'uniform'))

In [65]:
model = Sequential()

In [66]:
model.add(Dense(units = 40, input_dim = 7, 
                kernel_initializer = 'uniform', 
                bias_initializer='zeros', 
                activation = 'relu'))

In [67]:
model.add(Dense(units = 30, 
                kernel_initializer = 'uniform',
                bias_initializer='zeros', 
                activation = 'relu'))

In [68]:
model.add(Dense(units = 1, 
                kernel_initializer = 'uniform',
                activation = 'sigmoid'))

# 訓練模型

In [69]:
model.compile(loss = 'binary_crossentropy', 
              optimizer = 'adam', 
              metrics=['accuracy'])

In [70]:
model.fit(X_train, y_train, validation_split=0.1, epochs=500, batch_size=30, verbose=2)

Train on 801 samples, validate on 90 samples
Epoch 1/500
 - 2s - loss: 0.6820 - acc: 0.6105 - val_loss: 0.6707 - val_acc: 0.6222
Epoch 2/500
 - 0s - loss: 0.6610 - acc: 0.6155 - val_loss: 0.6581 - val_acc: 0.6222
Epoch 3/500
 - 0s - loss: 0.6539 - acc: 0.6155 - val_loss: 0.6492 - val_acc: 0.6222
Epoch 4/500
 - 0s - loss: 0.6446 - acc: 0.6155 - val_loss: 0.6324 - val_acc: 0.6222
Epoch 5/500
 - 0s - loss: 0.6215 - acc: 0.6180 - val_loss: 0.5955 - val_acc: 0.6222
Epoch 6/500
 - 0s - loss: 0.5782 - acc: 0.6879 - val_loss: 0.5304 - val_acc: 0.7778
Epoch 7/500
 - 0s - loss: 0.5229 - acc: 0.7903 - val_loss: 0.4796 - val_acc: 0.7889
Epoch 8/500
 - 0s - loss: 0.4993 - acc: 0.7865 - val_loss: 0.4676 - val_acc: 0.7889
Epoch 9/500
 - 0s - loss: 0.4738 - acc: 0.7965 - val_loss: 0.4575 - val_acc: 0.7889
Epoch 10/500
 - 0s - loss: 0.4754 - acc: 0.7978 - val_loss: 0.4481 - val_acc: 0.7889
Epoch 11/500
 - 0s - loss: 0.4603 - acc: 0.7990 - val_loss: 0.4474 - val_acc: 0.7889
Epoch 12/500
 - 0s - loss: 0.

Epoch 97/500
 - 0s - loss: 0.4101 - acc: 0.8215 - val_loss: 0.3512 - val_acc: 0.8556
Epoch 98/500
 - 0s - loss: 0.4093 - acc: 0.8265 - val_loss: 0.3465 - val_acc: 0.8667
Epoch 99/500
 - 0s - loss: 0.4106 - acc: 0.8277 - val_loss: 0.3517 - val_acc: 0.8556
Epoch 100/500
 - 0s - loss: 0.4103 - acc: 0.8202 - val_loss: 0.3494 - val_acc: 0.8444
Epoch 101/500
 - 0s - loss: 0.4129 - acc: 0.8240 - val_loss: 0.3506 - val_acc: 0.8444
Epoch 102/500
 - 0s - loss: 0.4128 - acc: 0.8227 - val_loss: 0.3475 - val_acc: 0.8667
Epoch 103/500
 - 0s - loss: 0.4064 - acc: 0.8315 - val_loss: 0.3487 - val_acc: 0.8667
Epoch 104/500
 - 0s - loss: 0.4065 - acc: 0.8240 - val_loss: 0.3473 - val_acc: 0.8556
Epoch 105/500
 - 0s - loss: 0.4120 - acc: 0.8240 - val_loss: 0.3468 - val_acc: 0.8556
Epoch 106/500
 - 0s - loss: 0.4084 - acc: 0.8302 - val_loss: 0.3499 - val_acc: 0.8333
Epoch 107/500
 - 0s - loss: 0.4139 - acc: 0.8215 - val_loss: 0.3474 - val_acc: 0.8556
Epoch 108/500
 - 0s - loss: 0.4048 - acc: 0.8327 - val_lo

Epoch 193/500
 - 0s - loss: 0.4014 - acc: 0.8315 - val_loss: 0.3462 - val_acc: 0.8667
Epoch 194/500
 - 0s - loss: 0.4026 - acc: 0.8277 - val_loss: 0.3442 - val_acc: 0.8333
Epoch 195/500
 - 0s - loss: 0.4154 - acc: 0.8190 - val_loss: 0.3691 - val_acc: 0.8556
Epoch 196/500
 - 0s - loss: 0.4081 - acc: 0.8265 - val_loss: 0.3471 - val_acc: 0.8667
Epoch 197/500
 - 0s - loss: 0.4019 - acc: 0.8290 - val_loss: 0.3482 - val_acc: 0.8667
Epoch 198/500
 - 0s - loss: 0.4034 - acc: 0.8277 - val_loss: 0.3465 - val_acc: 0.8667
Epoch 199/500
 - 0s - loss: 0.4060 - acc: 0.8252 - val_loss: 0.3449 - val_acc: 0.8667
Epoch 200/500
 - 0s - loss: 0.4071 - acc: 0.8290 - val_loss: 0.3459 - val_acc: 0.8667
Epoch 201/500
 - 0s - loss: 0.4016 - acc: 0.8290 - val_loss: 0.3460 - val_acc: 0.8444
Epoch 202/500
 - 0s - loss: 0.4027 - acc: 0.8265 - val_loss: 0.3450 - val_acc: 0.8667
Epoch 203/500
 - 0s - loss: 0.4030 - acc: 0.8290 - val_loss: 0.3474 - val_acc: 0.8667
Epoch 204/500
 - 0s - loss: 0.4025 - acc: 0.8252 - val

Epoch 289/500
 - 0s - loss: 0.3992 - acc: 0.8290 - val_loss: 0.3459 - val_acc: 0.8667
Epoch 290/500
 - 0s - loss: 0.4009 - acc: 0.8315 - val_loss: 0.3430 - val_acc: 0.8556
Epoch 291/500
 - 0s - loss: 0.3947 - acc: 0.8302 - val_loss: 0.3506 - val_acc: 0.8667
Epoch 292/500
 - 0s - loss: 0.3996 - acc: 0.8265 - val_loss: 0.3525 - val_acc: 0.8667
Epoch 293/500
 - 0s - loss: 0.3931 - acc: 0.8352 - val_loss: 0.3429 - val_acc: 0.8333
Epoch 294/500
 - 0s - loss: 0.4044 - acc: 0.8252 - val_loss: 0.3540 - val_acc: 0.8667
Epoch 295/500
 - 0s - loss: 0.3982 - acc: 0.8277 - val_loss: 0.3480 - val_acc: 0.8667
Epoch 296/500
 - 0s - loss: 0.4023 - acc: 0.8227 - val_loss: 0.3456 - val_acc: 0.8667
Epoch 297/500
 - 0s - loss: 0.3977 - acc: 0.8327 - val_loss: 0.3487 - val_acc: 0.8667
Epoch 298/500
 - 0s - loss: 0.4037 - acc: 0.8327 - val_loss: 0.3467 - val_acc: 0.8667
Epoch 299/500
 - 0s - loss: 0.4031 - acc: 0.8277 - val_loss: 0.3448 - val_acc: 0.8667
Epoch 300/500
 - 0s - loss: 0.3977 - acc: 0.8315 - val

Epoch 385/500
 - 0s - loss: 0.3941 - acc: 0.8290 - val_loss: 0.3426 - val_acc: 0.8444
Epoch 386/500
 - 0s - loss: 0.3946 - acc: 0.8265 - val_loss: 0.3425 - val_acc: 0.8444
Epoch 387/500
 - 0s - loss: 0.3943 - acc: 0.8215 - val_loss: 0.3466 - val_acc: 0.8556
Epoch 388/500
 - 0s - loss: 0.3952 - acc: 0.8390 - val_loss: 0.3419 - val_acc: 0.8333
Epoch 389/500
 - 0s - loss: 0.3928 - acc: 0.8365 - val_loss: 0.3482 - val_acc: 0.8667
Epoch 390/500
 - 0s - loss: 0.3929 - acc: 0.8190 - val_loss: 0.3423 - val_acc: 0.8333
Epoch 391/500
 - 0s - loss: 0.3955 - acc: 0.8315 - val_loss: 0.3449 - val_acc: 0.8556
Epoch 392/500
 - 0s - loss: 0.3934 - acc: 0.8290 - val_loss: 0.3496 - val_acc: 0.8667
Epoch 393/500
 - 0s - loss: 0.3955 - acc: 0.8252 - val_loss: 0.3422 - val_acc: 0.8333
Epoch 394/500
 - 0s - loss: 0.3952 - acc: 0.8302 - val_loss: 0.3436 - val_acc: 0.8556
Epoch 395/500
 - 0s - loss: 0.3954 - acc: 0.8315 - val_loss: 0.3421 - val_acc: 0.8556
Epoch 396/500
 - 0s - loss: 0.3936 - acc: 0.8302 - val

Epoch 481/500
 - 0s - loss: 0.3893 - acc: 0.8315 - val_loss: 0.3456 - val_acc: 0.8444
Epoch 482/500
 - 0s - loss: 0.3877 - acc: 0.8340 - val_loss: 0.3434 - val_acc: 0.8444
Epoch 483/500
 - 0s - loss: 0.3868 - acc: 0.8302 - val_loss: 0.3460 - val_acc: 0.8333
Epoch 484/500
 - 0s - loss: 0.3932 - acc: 0.8290 - val_loss: 0.3481 - val_acc: 0.8333
Epoch 485/500
 - 0s - loss: 0.3888 - acc: 0.8352 - val_loss: 0.3433 - val_acc: 0.8444
Epoch 486/500
 - 0s - loss: 0.3886 - acc: 0.8315 - val_loss: 0.3438 - val_acc: 0.8333
Epoch 487/500
 - 0s - loss: 0.3877 - acc: 0.8377 - val_loss: 0.3485 - val_acc: 0.8556
Epoch 488/500
 - 0s - loss: 0.3872 - acc: 0.8352 - val_loss: 0.3454 - val_acc: 0.8556
Epoch 489/500
 - 0s - loss: 0.3863 - acc: 0.8427 - val_loss: 0.3496 - val_acc: 0.8556
Epoch 490/500
 - 0s - loss: 0.3997 - acc: 0.8315 - val_loss: 0.3536 - val_acc: 0.8667
Epoch 491/500
 - 0s - loss: 0.3890 - acc: 0.8340 - val_loss: 0.3527 - val_acc: 0.8667
Epoch 492/500
 - 0s - loss: 0.3926 - acc: 0.8315 - val

In [71]:
scores = model.evaluate(X_train, y_train)
scores[1]

891/891 [==============================] - 0s 35us/step


0.8338945000928928

# 預測

In [73]:
y_predict= model.predict(X_test)

In [74]:
y_predict = np.where(y_predict>0.9,1,0)

In [75]:
y_predict = y_predict.flatten()

In [76]:
y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

# 匯出

In [77]:
submission=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':y_predict})

In [78]:
submission.to_csv('submission9.csv', index=False)

# 上傳後為0.74相較於HW03的0.77較低